### load libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

### import the data we gathered from MSD Taste Profiles and Spotify Web API

In [3]:
msd_spot_df = pd.read_csv('taste_profiles_spotify.csv', index_col=0, header=0, dtype={'userID': str, 'songID': str, 
    'playcount': int, 'trackID': str, 'artist': str, 'title': str, 'trackID_spot': str, 'artistID_spot': str,
    'artistGenres': str, 'artistPop': int, 'trackPop': int, 'trackAcoustic': float, 'trackDanceable': float,
    'trackDurMS': int, 'trackEnergy': float, 'trackInstrum': float, 'trackKey': int, 'trackLoud': float, 'trackMode': int,
    'trackSpeech': float, 'trackTempo': float, 'trackVal': float, 'albumYear': int})

msd_spot_df

,userID,songID,playcount,trackID,artist,title,trackID_spot,artistID_spot,artistGenres,artistPop,trackPop,trackAcoustic,trackDanceable,trackDurMS,trackEnergy,trackInstrum,trackKey,trackLoud,trackMode,trackSpeech,trackTempo,trackVal,albumYear
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,TRIRLYL128F42539D1,Billy Preston,Nothing from Nothing,72PoJMDfdaw9gGECgA9kTZ,0IecGJbdBeYSOVtSPRehh5,"classic soul, funk, psychedelic soul, quiet storm, rock keyboard, soul, southern soul",53,58,0.613000,0.808,156200,0.830,0.011200,9,-3.995,0,0.0283,102.800,0.909,1974
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,TRMHBXZ128F4238406,Paco De Lucia,Entre Dos Aguas,5hVqZ5hnvQGT4LOpUVuJUa,3h8OjAdgYXVRoMZ8jFd6Uw,"flamenco, flamenco guitar, rumba",55,39,0.923000,0.461,360426,0.654,0.874000,4,-12.673,0,0.0349,102.814,0.924,1997
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,TRYQMNI128F147C1C7,Josh Rouse,Under Cold Blue Stars,4HWqzvmeiaddzCDkSQcNxf,38euc5ce2q2MMzaoJcPstz,alternative country,44,13,0.023600,0.613,196333,0.692,0.000008,1,-6.880,1,0.0266,100.614,0.523,2002
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2,TRPLAXZ128F4292406,Jorge Drexler,12 segundos de oscuridad,5GCB9mqJzKN4kDKvmqQ03X,4ssUf5gLb1GBLxi1BhPrVt,"cantautor, latin afrobeat, latin alternative, rock en espanol",68,54,0.651000,0.749,164946,0.423,0.001160,7,-9.239,1,0.0302,78.005,0.507,2006
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,TRHNCIR128F42334A5,Kanye West,Stronger,0j2T0R9dR9qdJYsB7ciXhf,5K4W6rqBFWDnAN6FQUkS6x,"chicago rap, rap",92,77,0.005640,0.617,311866,0.717,0.000000,10,-7.858,0,0.1530,103.992,0.490,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10633750,cf8289419383259189afe6bb50c5115fd84f1064,SOHKQSO12A8AE47C9C,10,TRPLVWY12903CC08D3,Everything But The Girl,Time After Time,5RxNm1sQygVKpY9hEOvid5,13ccXrK7AmXb4TddMkE7jy,"electronica, new wave pop, pop rock, sophisti-pop, trip hop",60,37,0.899000,0.584,269360,0.203,0.000006,0,-13.612,1,0.0323,120.370,0.326,1992
10633751,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOABJBU12A8C13F63F,2,TRUDZCV128F42811F6,Cage The Elephant,Back Against The Wall,0vz64VTiPPBpcmla0QvAI9,26T3LtbuGT1Fu9m0eRq5X3,"modern rock, punk blues, rock",75,55,0.000643,0.598,228320,0.743,0.000000,1,-6.163,1,0.0305,110.334,0.534,2009
10633752,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOEISDE12A8AE4632E,1,TRKIBVN128F42283F4,King Kobra,Only The Strong Will Survive,5sUy4vaMwn4YCMPlr0RW0g,60DhbkIV5C2aLGgtpfUw2C,"glam metal, hard rock, sleaze rock",36,7,0.195000,0.460,240372,0.853,0.000000,4,-4.691,1,0.0328,91.632,0.550,1986
10633753,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOHHKGO12AC3DF57BF,1,TRODVNR12903CE6C50,Close Your Eyes,17:20,0IUhBvxsm2pLz6rzEM3YUn,4EJdKdDfQ0XQ4YBhh12xZe,christian hardcore,34,14,0.000005,0.203,136920,0.899,0.089900,11,-4.449,1,0.1680,186.031,0.775,2010


### get "user" (listener) and genre information

For our data, we will treat each user as a "transaction" with the "items" being the different genres of the 10+ songs they have listened to...

There are other ways this could be analyzed in the data as well. For example the "items" could be the artist or song names. However I'm concerned that there would be too much variability within these categories in the dataset for any significant rules to be found. So let's start with the genres:

In [4]:
msd_spot_arm_df = msd_spot_df[['userID', 'artistGenres']]
msd_spot_arm_df.head()

,userID,artistGenres
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"classic soul, funk, psychedelic soul, quiet storm, rock keyboard, soul, southern soul"
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"flamenco, flamenco guitar, rumba"
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,alternative country
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"cantautor, latin afrobeat, latin alternative, rock en espanol"
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"chicago rap, rap"


reduce the genre column to just one entry per observation:

In [94]:
# key terms were found during clustering prep, see that notebook for more details

key_terms = [
    'rock',
    'pop',
    'metal',
    'indie',
    'hip hop',
    'dance',
    'punk',
    'rap',
    'country',
    'blues',
    'folk',
    'latin',
    'jazz',
    'house',
    'soul',
    'rave',
    'funk',
    'r&b',
    'singer',
    'reggae',
    'chamber',
    'hardcore',
    'new age',
    'video game',
    'downtempo',
    'soundtrack',
    'trance',
    'dub',
    'tango',
    'rumba',
    'karaoke',
    'bluegrass',
    'lounge',
    'break',
    'ambient',
    'techno',
    'flamenco',
    'world',
    'industr',
    'electro',
    'nintendo',
    'neo mellow',
    'easy',
    'stomp',
    'celtic',
    'chill',
    'glitch',
    'jam',
    'lo-fi',
    'balkan',
    'melanch',
    'surf',
    'adult',
    'bossa nova',
    'tronica',
    'disco',
    'hardstyle',
    'salsa',
    'samba',
    'ska',
    'emo',
    'gospel',
    'vocaloid',
    'shojo',
    'grindcore',
    'hip-hop',
    'cappella',
    'big band',
    'hands up',
    'laboratorio',
    'bass',
    'grunge',
    'fiddle',
    'string',
    'banjo',
    'lullaby',
    'new wave',
    'dark wave',
    'children',
    'choir',
    'broadway',
    'classical',
    'folclore',
    'hollywood',
    'wonky',
    'trova',
    'electronica'
]

In [60]:
msd_spot_arm_df['items'] = ''
msd_spot_arm_df.head()

C:\Users\sophi\AppData\Local\Temp\ipykernel_14700\28392897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msd_spot_arm_df['items'] = ''


,userID,artistGenres,items
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"classic soul, funk, psychedelic soul, quiet storm, rock keyboard, soul, southern soul",
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"flamenco, flamenco guitar, rumba",
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,alternative country,
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"cantautor, latin afrobeat, latin alternative, rock en espanol",
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"chicago rap, rap",


In [7]:
len(msd_spot_arm_df)

10633755

In [8]:
# remove any rows with missing genre info
msd_spot_arm_df.dropna(inplace=True)
len(msd_spot_arm_df)

C:\Users\sophi\AppData\Local\Temp\ipykernel_14700\3087699006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msd_spot_arm_df.dropna(inplace=True)


10381467

In [9]:
msd_spot_arm_df.reset_index(inplace=True)

In [10]:
msd_spot_arm_df.drop(columns=['index'], inplace=True)

C:\Users\sophi\AppData\Local\Temp\ipykernel_14700\1402939672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msd_spot_arm_df.drop(columns=['index'], inplace=True)


In [62]:
# find the most frequent key term for each genre entry, and use that

for i,row in msd_spot_arm_df.iterrows():
    if len(row["artistGenres"].split(',')) == 1: #if there's only one genre listed...
        for w in key_terms: #find the corresponding key term and use that
            if w in row["artistGenres"]:
                msd_spot_arm_df.loc[i,"items"] = w

    else: #if there's more than one genre listed...
        g = row["artistGenres"].replace(',','')
        g = g.split(' ')
        t = max(set(g), key = g.count) #find the most frequent term used
        while t not in key_terms: #check that it's a key term
            g = set(g)
            g.discard(t) #if not get rid of it and try again
            g = list(g)
            if g == []: #if we run out of words
                t = '' #then quit and use empty string
                break
            else: t = max(set(g), key = g.count)
                
        msd_spot_arm_df.loc[i,"items"] = t
        
    if i%10000 == 0: print("Row", i)

C:\Users\sophi\AppData\Local\Temp\ipykernel_14700\101467698.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msd_spot_arm_df.loc[i,"items"] = t
C:\Users\sophi\AppData\Local\Temp\ipykernel_14700\101467698.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msd_spot_arm_df.loc[i,"items"] = w


Row 0
Row 10000
Row 20000
Row 30000
Row 40000
Row 50000
Row 60000
Row 70000
Row 80000
Row 90000
Row 100000
Row 110000
Row 120000
Row 130000
Row 140000
Row 150000
Row 160000
Row 170000
Row 180000
Row 190000
Row 200000
Row 210000
Row 220000
Row 230000
Row 240000
Row 250000
Row 260000
Row 270000
Row 280000
Row 290000
Row 300000
Row 310000
Row 320000
Row 330000
Row 340000
Row 350000
Row 360000
Row 370000
Row 380000
Row 390000
Row 400000
Row 410000
Row 420000
Row 430000
Row 440000
Row 450000
Row 460000
Row 470000
Row 480000
Row 490000
Row 500000
Row 510000
Row 520000
Row 530000
Row 540000
Row 550000
Row 560000
Row 570000
Row 580000
Row 590000
Row 600000
Row 610000
Row 620000
Row 630000
Row 640000
Row 650000
Row 660000
Row 670000
Row 680000
Row 690000
Row 700000
Row 710000
Row 720000
Row 730000
Row 740000
Row 750000
Row 760000
Row 770000
Row 780000
Row 790000
Row 800000
Row 810000
Row 820000
Row 830000
Row 840000
Row 850000
Row 860000
Row 870000
Row 880000
Row 890000
Row 900000
Row 910000
Ro

Row 6930000
Row 6940000
Row 6950000
Row 6960000
Row 6970000
Row 6980000
Row 6990000
Row 7000000
Row 7010000
Row 7020000
Row 7030000
Row 7040000
Row 7050000
Row 7060000
Row 7070000
Row 7080000
Row 7090000
Row 7100000
Row 7110000
Row 7120000
Row 7130000
Row 7140000
Row 7150000
Row 7160000
Row 7170000
Row 7180000
Row 7190000
Row 7200000
Row 7210000
Row 7220000
Row 7230000
Row 7240000
Row 7250000
Row 7260000
Row 7270000
Row 7280000
Row 7290000
Row 7300000
Row 7310000
Row 7320000
Row 7330000
Row 7340000
Row 7350000
Row 7360000
Row 7370000
Row 7380000
Row 7390000
Row 7400000
Row 7410000
Row 7420000
Row 7430000
Row 7440000
Row 7450000
Row 7460000
Row 7470000
Row 7480000
Row 7490000
Row 7500000
Row 7510000
Row 7520000
Row 7530000
Row 7540000
Row 7550000
Row 7560000
Row 7570000
Row 7580000
Row 7590000
Row 7600000
Row 7610000
Row 7620000
Row 7630000
Row 7640000
Row 7650000
Row 7660000
Row 7670000
Row 7680000
Row 7690000
Row 7700000
Row 7710000
Row 7720000
Row 7730000
Row 7740000
Row 7750000
Row 

In [64]:
msd_spot_arm_df.iloc[2000:2010]

,userID,artistGenres,items
2000,eda12b54342243175dba3db83f66eade127d2b4e,"bolero, cancion melodica, latin pop, musica yucateca, velha guarda",pop
2001,eda12b54342243175dba3db83f66eade127d2b4e,"electric bass, instrumental funk, jazz funk, jazz fusion, modern funk",funk
2002,f608c215606e6421a429ea28ad08243241d5347d,"electronica argentina, latin alternative, latin pop, pop electronico, rock en espanol",pop
2003,f608c215606e6421a429ea28ad08243241d5347d,"healing, new age",
2004,f608c215606e6421a429ea28ad08243241d5347d,"contemporary country, country, country road, oklahoma country",country
2005,f608c215606e6421a429ea28ad08243241d5347d,"country road, redneck",country
2006,f608c215606e6421a429ea28ad08243241d5347d,"acoustic blues, blues, blues rock, chicago blues, country blues, delta blues, harmonica blues, traditional blues",blues
2007,f608c215606e6421a429ea28ad08243241d5347d,"alternative metal, canadian rock, nu metal, post-grunge",metal
2008,f608c215606e6421a429ea28ad08243241d5347d,"metalcore, north carolina emo, screamo",emo
2009,f608c215606e6421a429ea28ad08243241d5347d,"contemporary country, country, country road",country


In [ ]:
# second pass to catch the two-word terms

for i,row in msd_spot_arm_df[msd_spot_arm_df["items"]==''].iterrows():
    g = row["artistGenres"].split(',')
    t = max(set(g), key = g.count) #find the most frequent term used
    while t.strip() not in key_terms: #check that it's a key term
        g = set(g)
        g.discard(t) #if not get rid of it and try again
        g = list(g)
        if g == []: #if we run out of words
            t = '' #then quit and use empty string
            break
        else: t = max(set(g), key = g.count)

    msd_spot_arm_df.loc[i,"items"] = t.strip()
    
    if i%2 == 0: print(i)

In [96]:
msd_spot_arm_df[msd_spot_arm_df["items"]=='']

,userID,artistGenres,items
38,4bd88bfb25263a75bbdd467e74018f4ae570e5df,"crunk, electropowerpop, screamo",
40,4bd88bfb25263a75bbdd467e74018f4ae570e5df,"crunk, electropowerpop, screamo",
47,e006b1a48f466bf59feefed32bec6494495a4436,uk garage,
81,17aa9f6dbdf753831da8f38c71b66b64373de613,"ectofolk, lilith, ""womens music""",
85,17aa9f6dbdf753831da8f38c71b66b64373de613,"alternative hip hop, east coast hip hop, turntablism",
...,...,...,...
10381334,be9261f70995b93001fb60427fd0674d285a0a0a,gothic americana,
10381337,be9261f70995b93001fb60427fd0674d285a0a0a,gothic americana,
10381371,a6c32fb03a210c135bd944b9b35849fd63a7765e,chanson,
10381419,8305c896f42308824da7d4386f4b9ee584281412,"manguebeat, mpb, nova musica pernambucana",


We were able to get the majority of the dataset this way. We're left with ~ 3% that could not be translated using these methods, mostly because the terms used are more rare/niche. The methods could be refined to capture more of these observations. But for now let's just proceed without them for simplicity.

In [98]:
msd_spot_arm_df = msd_spot_arm_df[msd_spot_arm_df["items"]!='']
msd_spot_arm_df

,userID,artistGenres,items
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"classic soul, funk, psychedelic soul, quiet storm, rock keyboard, soul, southern soul",soul
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"flamenco, flamenco guitar, rumba",flamenco
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,alternative country,country
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"cantautor, latin afrobeat, latin alternative, rock en espanol",latin
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,"chicago rap, rap",rap
...,...,...,...
10381461,cf8289419383259189afe6bb50c5115fd84f1064,"british indie rock, english indie rock",rock
10381462,cf8289419383259189afe6bb50c5115fd84f1064,"electronica, new wave pop, pop rock, sophisti-pop, trip hop",pop
10381463,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,"modern rock, punk blues, rock",rock
10381464,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,"glam metal, hard rock, sleaze rock",rock


In [ ]:
msd_spot_arm_df.reset_index(inplace=True)
msd_spot_arm_df.drop(columns=['index'], inplace=True)

In [104]:
msd_spot_arm_df.drop(columns=['artistGenres'], inplace=True)
msd_spot_arm_df.head()

C:\Users\sophi\AppData\Local\Temp\ipykernel_14700\2159814041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msd_spot_arm_df.drop(columns=['artistGenres'], inplace=True)


,userID,items
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,soul
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,flamenco
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,country
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,latin
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,rap


### group by user to create "transactions"

In [108]:
transact_df = msd_spot_arm_df.groupby('userID')['items'].apply(list)

In [113]:
transact_df = transact_df.reset_index(name='items')

In [114]:
transact_df

,userID,items
0,00001638d6189236866af9bbf309ae6c2347ffdc,"[indie, pop, pop, pop]"
1,0000175652312d12576d9e6b84f600caa24c4715,"[rock, trance, rock, indie, rock]"
2,00003a4459f33b92906be11abe0e93efc423c0ff,"[trance, rock, metal]"
3,00004fb90a86beb8bed1e9e328f5d9b6ee7dc03e,"[rock, rock, rock, rock, rock, rock, rock, rock, rock, rock, rock, rock]"
4,000060ca4e6bea0a5c9037fc1bbd7bbabb98c754,"[rock, metal, blues, pop]"
...,...,...
879240,ffffdc274ca76d154b4e56b2dbc82ff538c93c0b,"[rock, rock, pop, rock, blues]"
879241,ffffdc6c89988cd6119067769162948eacf8b670,"[rock, latin, pop, pop, pop, metal, rock]"
879242,fffff3e690fcda840b716ce7249d8935ff3323fc,"[rock, pop, r&b, samba]"
879243,fffff67d54a40927c93d03bd6c816b034b59f087,"[rock, rock, blues, rock, jazz, jazz, jazz, rock, rock, jazz, jazz, jazz, jazz, jazz, rock, jazz, rock, rock, jazz, jazz]"


get rid of duplicate items

In [118]:
for i,row in transact_df.iterrows():
    transact_df.loc[i,"items"] = list(set(transact_df.loc[i,"items"]))

In [119]:
transact_df

,userID,items
0,00001638d6189236866af9bbf309ae6c2347ffdc,"[indie, pop]"
1,0000175652312d12576d9e6b84f600caa24c4715,"[rock, indie, trance]"
2,00003a4459f33b92906be11abe0e93efc423c0ff,"[rock, metal, trance]"
3,00004fb90a86beb8bed1e9e328f5d9b6ee7dc03e,[rock]
4,000060ca4e6bea0a5c9037fc1bbd7bbabb98c754,"[rock, blues, pop, metal]"
...,...,...
879240,ffffdc274ca76d154b4e56b2dbc82ff538c93c0b,"[rock, blues, pop]"
879241,ffffdc6c89988cd6119067769162948eacf8b670,"[rock, latin, pop, metal]"
879242,fffff3e690fcda840b716ce7249d8935ff3323fc,"[rock, samba, pop, r&b]"
879243,fffff67d54a40927c93d03bd6c816b034b59f087,"[rock, blues, jazz]"


In [ ]:
transact_df.to_csv('arm_data.csv')